<a href="https://colab.research.google.com/github/gbessardon/Land_Cover_comparison/blob/main/ECOSG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Running environment options

In [1]:
colab=1 # 1 running on google collab
drive=1 # 1 need to mount google drive

## LUCAS file location

In [2]:
fnshp='/content/drive/MyDrive/LUCAS2018/LUCAS_2018_Copernicus_polygons.shp'
fncsv='/content/drive/MyDrive/LUCAS2018/LUCAS_2018_Copernicus_attributes.csv'

## ECOSG location

In [3]:
fnameSG='/content/drive/MyDrive/ECOCLIMAP_SG/ecosg_final_map.dir' #ecosg filename
SGfolder=fnameSG.replace('ecosg_final_map.dir','')

# Run on collab and drive 
set colab=1 and drive=1  in running enviroment options

## MOUNT google drive

In [4]:
if drive==1:
  from google.colab import drive
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## install geopandas and pyrpoj for collab

In [ ]:
if colab==1:
  !pip install geopandas
  !pip install pyproj
  !pip install rioxarray

# import libraires

In [ ]:
import os
from osgeo import gdal
from shapely import geometry
import geopandas as gpd
import numpy as np
import shutil

# Create functions

## create directory to store the segment before zipping

In [ ]:
def create_sgement_dir(path=os.getcwd(),ending='LUCAS_Segment'):
    outputdir=os.path.join(path,ending)
    if not os.path.isdir(outputdir):
      os.mkdir(outputdir)
    return outputdir

## CUT ECOSG segments

In [ ]:
def cutSG(fnSG,bounds,outfn):
  #Crop to a sligthly larger area
  lonmin=bounds[0]-0.1
  latmin=bounds[1]-0.1
  lonmax=bounds[2]+0.1
  latmax=bounds[3]+0.1
  temp_fn='ECOSGcut.tif' 
  src_fn=fnameSG
  ds=gdal.Open(src_fn)
  gdal.Translate(temp_fn,ds,
               projWin=[lonmin,latmax,lonmax,latmin], 
               projWinSRS='EPSG:4326')

  # cut it and adjust to the resolution
  ds=gdal.Open(temp_fn)
  gt=ds.GetGeoTransform() 
  # Recrop to the actual area and increase the resolution  
  opt=gdal.WarpOptions(dstSRS="EPSG:4326",
                       outputBounds=bounds,xRes=gt[1]/30,yRes=gt[1]/30)
  gdal.Warp(outfn,temp_fn,options=opt)
  return

# Main

## Open LUCAS shapefile in a geodatabase

In [ ]:
LUCASgdf = gpd.read_file(fnshp)

## Create directory to store the segment before zipping

In [ ]:
outputdir=create_sgement_dir(path=os.getcwd(),ending='LUCAS_Segment')

## Create CORINE segments corresponding to the LUCAS segments

In [ ]:
for i,d in LUCASgdf.iterrows():
      lucasp=LUCASgdf['geometry'].loc[i]
      bounds=lucasp.bounds
      lucasid=LUCASgdf['POINT_ID'].loc[i]
      outfile=os.path.join(outputdir,str(lucasid)+'.tif')
      cutSG(fnameSG,bounds,outfn)

## Save in a zip archive in google drive (prevents I/O errors)

In [ ]:
shutil.make_archive(os.path.join(SGfolder,'LUCAS_Segment'), 'zip', outputdir)